In [ ]:
# import spotipy library
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import os 
import pandas as pd

In [ ]:
# setting up the credentials
# Create app API in developer.spotify.com
os.environ['SPOTIPY_CLIENT_ID'] = {YOUR_CLIENT_ID}
os.environ['SPOTIPY_CLIENT_SECRET'] = {YOUR_CLIENT_SECRET}
os.environ['SPOTIPY_REDIRECT_URI'] = {YOUR_REDIRECT_URL}

In [ ]:
# Auth to using user API using credential Auth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
user_id = sp.me()['id']


In [ ]:
# sp.current_user_follow_playlist()

In [ ]:
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [ ]:
all_track_ids = []

In [ ]:
all_in = get_playlist_tracks('6y3UDKRhVtfa4hAKCszqNl')
# new_jeans_mix = get_playlist_tracks('37i9dQZF1EIXtL62omEoXC')
# creepy_nuts_mix = get_playlist_tracks('37i9dQZF1EIXtL62omEoXC')

In [ ]:
def input_track_to_all_track_ids(current_mix):
    global all_track_ids
    for i in current_mix:
        all_track_ids.append(i['track']['id'])

In [ ]:
# append to all_track_ids
input_track_to_all_track_ids(all_in)
# input_track_to_all_track_ids(creepy_nuts_mix)

In [ ]:
# all_track_ids[49]
len(all_track_ids)
# all_track_ids

In [ ]:
# sp.audio_features('0pHylQR53epYtRcVIhUSCh')
# sp.audio_analysis('0pHylQR53epYtRcVIhUSCh')
# sp.track('70t7Q6AYG6ZgTYmJWcnkUM')

In [ ]:
def get_track_features(id):
    metadata = sp.track(id)
    features = sp.audio_features(id)

    # metadata
    track_id = id
    name = metadata['name']
    album = metadata['album']['name']
    artist = metadata['album']['artists'][0]['name']
    release_date = metadata['album']['release_date']
    length = metadata['duration_ms']
    popularity = metadata['popularity']

    # audio features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [track_id, name, album, artist, release_date, length, popularity, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

In [ ]:
tracks = []

In [ ]:
min = 0
max = 50
for i in range(min,max):
  track = get_track_features(all_track_ids[i])
  tracks.append(track)

len(tracks)

In [ ]:
# tracks[1335]
# len(tracks)

In [ ]:
df = pd.DataFrame(tracks, columns = ['track_id','name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature'])
df['name'] = df['name'].astype("string")
df = df.dropna().reset_index()
df['is_id'] = ""

In [ ]:
def load_lst_file(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:
        # Read each line, strip whitespace, and filter out words with length < 3
        words_list = [line.strip() for line in file if len(line.strip()) >= 3]
    return words_list

# Usage example
file_path = 'indonesian-wordlist.lst'
indonesian_keywords = load_lst_file(file_path)
len(indonesian_keywords)

In [ ]:
for i in range(len(df)):
    flag = "no"
    for title in df.loc[i,'name'].lower().split():
        for keyword in indonesian_keywords: 
            if title == keyword.lower():
                df.loc[i,'is_id'] = True
                flag = "yes"
                break
    if flag == "no":
        df.loc[i,'is_id'] = False

df.loc[df['is_id'] == True]

In [ ]:
# df.to_excel('all_in_playlist.xlsx', index = False)
df = pd.read_excel('all_in_playlist.xlsx')

In [ ]:
# input_tracks = df.loc[df['energy'] > 0.5].sort_values(by='release_date',ascending= False)['track_id'].drop_duplicates().tolist()
input_tracks = df.loc[df['is_id'] == True].sort_values(by='release_date',ascending= False)['track_id'].drop_duplicates().tolist()
len(input_tracks)

In [ ]:
sp.user_playlist_create(user_id, 'indo true', public=True, collaborative=False, description='python filter by title for indonesian song from `all in` playlist')

In [ ]:
print(len(input_tracks))
min_iloc = 0
max_iloc = 100
sp.playlist_add_items('3RjAmLKOxlwiRLSE8lTmSg', input_tracks[min_iloc:max_iloc])

indo_true = get_playlist_tracks('3RjAmLKOxlwiRLSE8lTmSg')
print(len(indo_true))